# Введение в обработку естественного языка

## Урок 1. Предобработка текста

### Практическое задание

Осуществим предобработку данных с Твиттера, чтобы очищенные данные в дальнейшем
использовать для задачи классификации. Данный датасет содержит негативные (label = 1)
и нейтральные (label = 0) высказывания. Для работы объединим train_df и test_df.

Задания:
1. Удалим @user из всех твитов с помощью паттерна "@[\w]*". Для этого создадим
функцию:
● для того, чтобы найти все вхождения паттерна в тексте, необходимо
использовать re.findall(pattern, input_txt)
● для для замены @user на пробел, необходимо использовать re.sub()
2. Изменим регистр твитов на нижний с помощью .lower().
3. Заменим сокращения с апострофами (пример: ain't, can't) на пробел, используя
apostrophe_dict. Для этого необходимо сделать функцию: для каждого слова в
тексте проверить (for word in text.split()), если слово есть в словаре apostrophe_dict в
качестве ключа (сокращенного слова), то заменить ключ на значение (полную
версию слова).
4. Заменим сокращения на их полные формы, используя short_word_dict. Для этого
воспользуемся функцией, используемой в предыдущем пункте.
5. Заменим эмотиконы (пример: ":)" = "happy") на пробелы, используя emoticon_dict.
Для этого воспользуемся функцией, используемой в предыдущем пункте.
6. Заменим пунктуацию на пробелы, используя re.sub() и паттерн r'[^\w\s]'.
7. Заменим спец. символы на пробелы, используя re.sub() и паттерн r'[^a-zA-Z0-9]'.
8. Заменим числа на пробелы, используя re.sub() и паттерн r'[^a-zA-Z]'.
9. Удалим из текста слова длиной в 1 символ, используя ' '.join([w for w in x.split() if
len(w)>1]).
10. Поделим твиты на токены с помощью nltk.tokenize.word_tokenize, создав новый
столбец 'tweet_token'.
11. Удалим стоп-слова из токенов, используя nltk.corpus.stopwords. Создадим столбец
'tweet_token_filtered' без стоп-слов.
12. Применим стемминг к токенам с помощью nltk.stem.PorterStemmer. Создадим
столбец 'tweet_stemmed' после применения стемминга.
13. Применим лемматизацию к токенам с помощью
nltk.stem.wordnet.WordNetLemmatizer. Создадим столбец 'tweet_lemmatized' после
применения лемматизации.
14. Сохраним результат предобработки в pickle-файл.

#### Импорт библиотек

In [1]:
import numpy as np
import pandas as pd

import re

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

import pickle

#### Настройки проекта

In [2]:
# Путь к обучающей выборке.
PATH__DATA_TRAIN = r'train_tweets.csv'

# Путь к тестовой выборке.
PATH__DATA_TEST = r'test_tweets.csv'

# Путь к результату работы.
PATH__DATA_RESULT = r'df.plk'
PATH__DATA_RESULT_CSV = r'df.csv'

### Подготовка

In [3]:
# Загрузка обучающей выборки.
df_train = pd.read_csv(PATH__DATA_TRAIN)

In [4]:
# Проверка: вывод первых пяти строк обучающей выборки.
df_train.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [5]:
# Проверка: вывод основной информации о датасете обучающей выборки.
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31962 entries, 0 to 31961
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      31962 non-null  int64 
 1   label   31962 non-null  int64 
 2   tweet   31962 non-null  object
dtypes: int64(2), object(1)
memory usage: 749.2+ KB


In [6]:
# Загрузка тестовой выборки.
df_test = pd.read_csv(PATH__DATA_TEST)

In [7]:
# Проверка: вывод первых пяти строк тестовой выборки.
df_test.head()

,id,tweet
0,31963,#studiolife #aislife #requires #passion #dedic...
1,31964,@user #white #supremacists want everyone to s...
2,31965,safe ways to heal your #acne!! #altwaystohe...
3,31966,is the hp and the cursed child book up for res...
4,31967,"3rd #bihday to my amazing, hilarious #nephew..."


In [8]:
# Проверка: вывод основной информации о датасете тестовой выборки.
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17197 entries, 0 to 17196
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      17197 non-null  int64 
 1   tweet   17197 non-null  object
dtypes: int64(1), object(1)
memory usage: 268.8+ KB


In [9]:
# Объединение обучающей и тестовой выборок.
df = pd.concat([df_train, df_test])

In [10]:
# Проверка: вывод первых пяти строк общей выборки.
df.head()

,id,label,tweet
0,1,0.0,@user when a father is dysfunctional and is s...
1,2,0.0,@user @user thanks for #lyft credit i can't us...
2,3,0.0,bihday your majesty
3,4,0.0,#model i love u take with u all the time in ...
4,5,0.0,factsguide: society now #motivation


In [11]:
# Проверка: вывод основной информации о датасете общей выборки.
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49159 entries, 0 to 17196
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      49159 non-null  int64  
 1   label   31962 non-null  float64
 2   tweet   49159 non-null  object 
dtypes: float64(1), int64(1), object(1)
memory usage: 1.5+ MB


### Выполнение задания

#### Задание 1
Удалим @user из всех твитов с помощью паттерна "@[\w]*". Для этого создадим функцию:

    ● для того, чтобы найти все вхождения паттерна в тексте, необходимо использовать re.findall(pattern, input_txt)
    ● для для замены @user на пробел, необходимо использовать re.sub()

In [12]:
# Функция для удаления @user.
def func__remove_user(text):
    return re.sub('@[\w]*', '', text)

In [13]:
# Удаление @user.
df['tweet'] = df['tweet'].apply(lambda x: func__remove_user(x))

In [14]:
# Проверка: вывод первых пяти строк общей выборки.
df.head()

,id,label,tweet
0,1,0.0,when a father is dysfunctional and is so sel...
1,2,0.0,thanks for #lyft credit i can't use cause th...
2,3,0.0,bihday your majesty
3,4,0.0,#model i love u take with u all the time in ...
4,5,0.0,factsguide: society now #motivation


#### Задание 2
Изменим регистр твитов на нижний с помощью .lower()

In [15]:
# Изменение регистра твитов на нижний.
df['tweet'] = df['tweet'].str.lower()

In [16]:
# Проверка: вывод первых пяти строк общей выборки.
df.head()

,id,label,tweet
0,1,0.0,when a father is dysfunctional and is so sel...
1,2,0.0,thanks for #lyft credit i can't use cause th...
2,3,0.0,bihday your majesty
3,4,0.0,#model i love u take with u all the time in ...
4,5,0.0,factsguide: society now #motivation


#### Задание 3
Заменим сокращения с апострофами (пример: ain't, can't) на пробел, используя apostrophe_dict. Для этого необходимо сделать функцию: для каждого слова в тексте проверить (for word in text.split()), если слово есть в словаре apostrophe_dict в
качестве ключа (сокращенного слова), то заменить ключ на значение (полную версию слова).

In [17]:
# Словарь сокращений.
apostrophe_dict = {
    'don’t': 'do not',
    'it’s': 'it is',
    'isn’t': 'is not',
    'you’ve': 'you have',
    'can’t': 'cannot ',
    'you’ve': 'you have',
    'won’t': 'will not',
    'you’ve': 'you have',
    'ain\'t': 'are not'
}

In [18]:
# Функция для замены сокращений.
def func__replace_contraction(text, dict):
    list_text = text.split()
    list_text = [word if word not in dict.keys() else dict.get(word) for word in list_text]
    return ' '.join(list_text)

In [19]:
# Замена сокращений.
df['tweet'] = df['tweet'].apply(lambda x: func__replace_contraction(x, apostrophe_dict))

In [20]:
# Проверка: вывод первых пяти строк общей выборки.
df.head()

,id,label,tweet
0,1,0.0,when a father is dysfunctional and is so selfi...
1,2,0.0,thanks for #lyft credit i can't use cause they...
2,3,0.0,bihday your majesty
3,4,0.0,#model i love u take with u all the time in ur...
4,5,0.0,factsguide: society now #motivation


#### Задание 4
Заменим сокращения на их полные формы, используя short_word_dict. Для этого воспользуемся функцией, используемой в предыдущем пункте.

In [21]:
# Словарь сокращений.
short_word_dict = {
    'wanna': 'want',
    'gonna': 'going to',
    'gotta': 'have got to'
}

In [22]:
# Замена сокращений.
df['tweet'] = df['tweet'].apply(lambda x: func__replace_contraction(x, short_word_dict))

In [23]:
# Проверка: вывод первых пяти строк общей выборки.
df.head()

,id,label,tweet
0,1,0.0,when a father is dysfunctional and is so selfi...
1,2,0.0,thanks for #lyft credit i can't use cause they...
2,3,0.0,bihday your majesty
3,4,0.0,#model i love u take with u all the time in ur...
4,5,0.0,factsguide: society now #motivation


#### Задание 5
Заменим эмотиконы (пример: ":)" = "happy") на пробелы, используя emoticon_dict. Для этого воспользуемся функцией, используемой в предыдущем пункте.

In [24]:
# Словарь сокращений.
emoticon_dict = {
    ':)': 'happy',
    ':-)': 'happy',
    '=)': 'happy',
    ':(': 'unhappy',
    ':-(': 'unhappy',
    '=(': 'unhappy'
}

In [25]:
# Замена сокращений.
df['tweet'] = df['tweet'].apply(lambda x: func__replace_contraction(x, emoticon_dict))

In [26]:
# Проверка: вывод первых пяти строк общей выборки.
df.head()

,id,label,tweet
0,1,0.0,when a father is dysfunctional and is so selfi...
1,2,0.0,thanks for #lyft credit i can't use cause they...
2,3,0.0,bihday your majesty
3,4,0.0,#model i love u take with u all the time in ur...
4,5,0.0,factsguide: society now #motivation


#### Задание 6
Заменим пунктуацию на пробелы, используя re.sub() и паттерн r'[^\w\s]'.

In [27]:
# Функция для замены пунктуации.
def func__remove_punctuation(text):
    return re.sub(r'[^\w\s]', '', text)

In [28]:
# Замена пунктуации.
df['tweet'] = df['tweet'].apply(lambda x: func__remove_punctuation(x))

In [29]:
# Проверка: вывод первых пяти строк общей выборки.
df.head()

,id,label,tweet
0,1,0.0,when a father is dysfunctional and is so selfi...
1,2,0.0,thanks for lyft credit i cant use cause they d...
2,3,0.0,bihday your majesty
3,4,0.0,model i love u take with u all the time in urð...
4,5,0.0,factsguide society now motivation


#### Задание 7
Заменим спец. символы на пробелы, используя re.sub() и паттерн r'[^a-zA-Z0-9]'.

In [30]:
# Функция для замены спец. символов.
def func__remove_special_symbols(text):
    return re.sub(r'[^a-zA-Z0-9 ]', '', text)

In [31]:
# Замена спец. символов.
df['tweet'] = df['tweet'].apply(lambda x: func__remove_special_symbols(x))

In [32]:
# Проверка: вывод первых пяти строк общей выборки.
df.head()

,id,label,tweet
0,1,0.0,when a father is dysfunctional and is so selfi...
1,2,0.0,thanks for lyft credit i cant use cause they d...
2,3,0.0,bihday your majesty
3,4,0.0,model i love u take with u all the time in ur
4,5,0.0,factsguide society now motivation


#### Задание 8
Заменим числа на пробелы, используя re.sub() и паттерн r'[^a-zA-Z]'.

In [33]:
# Функция для замены спец. символов.
def func__remove_numerals(text):
    return re.sub(r'[^a-zA-Z ]', '', text)

In [34]:
# Замена спец. символов.
df['tweet'] = df['tweet'].apply(lambda x: func__remove_numerals(x))

In [35]:
# Проверка: вывод первых пяти строк общей выборки.
df.head()

,id,label,tweet
0,1,0.0,when a father is dysfunctional and is so selfi...
1,2,0.0,thanks for lyft credit i cant use cause they d...
2,3,0.0,bihday your majesty
3,4,0.0,model i love u take with u all the time in ur
4,5,0.0,factsguide society now motivation


#### Задание 9
Удалим из текста слова длиной в 1 символ, используя ' '.join([w for w in x.split() if len(w)>1]).

In [36]:
# Удаление из текста слов длинной в 1 символ.
df['tweet'] = df['tweet'].apply(lambda x: ' '.join([word for word in x.split() if len(word) > 1]))

In [37]:
# Проверка: вывод первых пяти строк общей выборки.
df.head()

,id,label,tweet
0,1,0.0,when father is dysfunctional and is so selfish...
1,2,0.0,thanks for lyft credit cant use cause they don...
2,3,0.0,bihday your majesty
3,4,0.0,model love take with all the time in ur
4,5,0.0,factsguide society now motivation


#### Задание 10
Поделим твиты на токены с помощью nltk.tokenize.word_tokenize, создав новый столбец 'tweet_token'.

In [38]:
%%time

# Деление твитов на токены.
df['tweet_token'] = df['tweet'].apply(lambda x: word_tokenize(x))

CPU times: total: 3.12 s
Wall time: 3.11 s


In [39]:
# Проверка: вывод первых пяти строк общей выборки.
df.head()

,id,label,tweet,tweet_token
0,1,0.0,when father is dysfunctional and is so selfish...,"[when, father, is, dysfunctional, and, is, so,..."
1,2,0.0,thanks for lyft credit cant use cause they don...,"[thanks, for, lyft, credit, cant, use, cause, ..."
2,3,0.0,bihday your majesty,"[bihday, your, majesty]"
3,4,0.0,model love take with all the time in ur,"[model, love, take, with, all, the, time, in, ur]"
4,5,0.0,factsguide society now motivation,"[factsguide, society, now, motivation]"


#### Задание 11
Удалим стоп-слова из токенов, используя nltk.corpus.stopwords. Создадим столбец 'tweet_token_filtered' без стоп-слов.

In [40]:
%%time

# Удаление стоп-слов.
df['tweet_token_filtered'] = df['tweet_token'].apply(lambda x: [word for word in x if word not in stopwords.words('english')])

CPU times: total: 1min 14s
Wall time: 1min 14s


In [41]:
# Проверка: вывод первых пяти строк общей выборки.
df.head()

,id,label,tweet,tweet_token,tweet_token_filtered
0,1,0.0,when father is dysfunctional and is so selfish...,"[when, father, is, dysfunctional, and, is, so,...","[father, dysfunctional, selfish, drags, kids, ..."
1,2,0.0,thanks for lyft credit cant use cause they don...,"[thanks, for, lyft, credit, cant, use, cause, ...","[thanks, lyft, credit, cant, use, cause, dont,..."
2,3,0.0,bihday your majesty,"[bihday, your, majesty]","[bihday, majesty]"
3,4,0.0,model love take with all the time in ur,"[model, love, take, with, all, the, time, in, ur]","[model, love, take, time, ur]"
4,5,0.0,factsguide society now motivation,"[factsguide, society, now, motivation]","[factsguide, society, motivation]"


#### Задание 12
Применим стемминг к токенам с помощью nltk.stem.PorterStemmer. Создадим cтолбец 'tweet_stemmed' после применения стемминга.

In [42]:
%%time

# Стемминг.
df['tweet_stemmed'] = df['tweet_token_filtered'].apply(lambda x: [PorterStemmer().stem(word) for word in x])

CPU times: total: 5.39 s
Wall time: 5.4 s


In [43]:
# Проверка: вывод первых пяти строк общей выборки.
df.head()

,id,label,tweet,tweet_token,tweet_token_filtered,tweet_stemmed
0,1,0.0,when father is dysfunctional and is so selfish...,"[when, father, is, dysfunctional, and, is, so,...","[father, dysfunctional, selfish, drags, kids, ...","[father, dysfunct, selfish, drag, kid, dysfunc..."
1,2,0.0,thanks for lyft credit cant use cause they don...,"[thanks, for, lyft, credit, cant, use, cause, ...","[thanks, lyft, credit, cant, use, cause, dont,...","[thank, lyft, credit, cant, use, caus, dont, o..."
2,3,0.0,bihday your majesty,"[bihday, your, majesty]","[bihday, majesty]","[bihday, majesti]"
3,4,0.0,model love take with all the time in ur,"[model, love, take, with, all, the, time, in, ur]","[model, love, take, time, ur]","[model, love, take, time, ur]"
4,5,0.0,factsguide society now motivation,"[factsguide, society, now, motivation]","[factsguide, society, motivation]","[factsguid, societi, motiv]"


#### Задание 13
Применим лемматизацию к токенам с помощью nltk.stem.wordnet.WordNetLemmatizer. Создадим столбец 'tweet_lemmatized' после применения лемматизации.

In [44]:
%%time

# Лемматизация.
df['tweet_lemmatized'] = df['tweet_token_filtered'].apply(lambda x: [WordNetLemmatizer().lemmatize(word) for word in x])

CPU times: total: 1.69 s
Wall time: 1.69 s


In [45]:
# Проверка: вывод первых пяти строк общей выборки.
df.head()

,id,label,tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
0,1,0.0,when father is dysfunctional and is so selfish...,"[when, father, is, dysfunctional, and, is, so,...","[father, dysfunctional, selfish, drags, kids, ...","[father, dysfunct, selfish, drag, kid, dysfunc...","[father, dysfunctional, selfish, drag, kid, dy..."
1,2,0.0,thanks for lyft credit cant use cause they don...,"[thanks, for, lyft, credit, cant, use, cause, ...","[thanks, lyft, credit, cant, use, cause, dont,...","[thank, lyft, credit, cant, use, caus, dont, o...","[thanks, lyft, credit, cant, use, cause, dont,..."
2,3,0.0,bihday your majesty,"[bihday, your, majesty]","[bihday, majesty]","[bihday, majesti]","[bihday, majesty]"
3,4,0.0,model love take with all the time in ur,"[model, love, take, with, all, the, time, in, ur]","[model, love, take, time, ur]","[model, love, take, time, ur]","[model, love, take, time, ur]"
4,5,0.0,factsguide society now motivation,"[factsguide, society, now, motivation]","[factsguide, society, motivation]","[factsguid, societi, motiv]","[factsguide, society, motivation]"


#### Задание 14
Сохраним результат предобработки в pickle-файл.

In [46]:
# Сохранение результата.
with open(PATH__DATA_RESULT, 'wb') as file:
    pickle.dump(obj=df, file=file)

In [47]:
# Сохранение результата.
df.to_csv(PATH__DATA_RESULT_CSV)